In [9]:
import pandas as pd
import numpy as np
import random


np.random.seed(42)

# Кол-во пользователей
USER_QUANT = 10000
# Кол-во айтемов
ITEM_QUANT = 100
# Мат ожидание кол-ва успещных взаимодействий
ACTION_QUANT = 8
# Размер одновременно отображаемых значений
WINDOW_SIZE = 6
# Вклад интересов пользователя
INTERSTS_SCORE = 0.4
# Вклад пола пользователя
SEX_TYPE_SCORE = 0.4
# Добавка за время
DATETIME_SCORE = 0.2
# Коэффициент сглаживания скора
POW_VAL = 0.7
# Возможные жанры
GENRES = ["Драма", "Комедия", "Триллер", "Мелодрама", "Фантастика"]
# Количество жанров - интересов пользователя
NUM_INTERESTS_GENRES = 3
# пол пользователя
SEX_TYPES = ["M", "F"]
# действия пользователя
ACTIONS = ['show', 'click']

# создаем список пользователей и айтемов
users = [str(i) for i in range(USER_QUANT)]
items = [str(i) for i in range(ITEM_QUANT)]


users_df = pd.DataFrame({
    'user_id': users,
    'sex_type': [random.choice(SEX_TYPES) for _ in range(USER_QUANT)],
    # Пользователю присваевается NUM_INTERESTS_GENRES скрытых интересов
    "interests": [random.sample(GENRES, k=NUM_INTERESTS_GENRES) for _ in range(USER_QUANT)],
})



items_df = pd.DataFrame({
    'item_id': items,
    'genre': [random.choice(GENRES)  for _ in range(ITEM_QUANT)],
    # Айтем может быть интересен представителем конкретного пола или не быть интересен никому
    "interests_sex": [random.choice(SEX_TYPES + [""])  for _ in range(ITEM_QUANT)],
    "mean_users_score": np.random.uniform(size=ITEM_QUANT) * 5
})


# Создадим датасет качества объектов
item_quality = items_df[['item_id', 'mean_users_score']]
item_quality["item_quality"] = item_quality['mean_users_score']
item_quality["item_quality"] = item_quality["item_quality"] / item_quality["item_quality"].sum()
item_quality_sorted = item_quality.sort_values(by='item_quality', ascending=False).reset_index(drop=True)
# Мы будем считать, что что бы долистать дор обхъекта нужно сдлеать несколко скроллов
item_quality_sorted["window"] = item_quality_sorted.index.values // WINDOW_SIZE

# Рассчитываем датасет с необходимыми скроллами
data_windows = pd.DataFrame(columns=["item_id", "item_showed"])
for i in range(item_quality_sorted["window"].max()):
    items_block = item_quality_sorted[item_quality_sorted["window"] == i][["item_id"]]
    showed_items = item_quality_sorted[item_quality_sorted["window"] <= i][["item_id"]].rename(columns={"item_id": "item_showed"})
    df_merged = pd.merge(items_block, showed_items, how='cross')
    data_windows = data_windows._append(df_merged)

# Оцениваем скоры релевантности
user_probs = pd.merge(users_df, item_quality_sorted, how="cross").merge(items_df, on="item_id")
user_probs['datetime'] = pd.to_datetime(np.random.rand(user_probs.shape[0]) * 20000000000000000) + pd.DateOffset(years=55)
user_probs['datetime_score'] = DATETIME_SCORE * ((user_probs['datetime'] - user_probs['datetime'].min()) / (user_probs['datetime'].max() - user_probs['datetime'].min()))
user_probs["score"] = user_probs.apply(lambda x: x['datetime_score'] + x["item_quality"] + INTERSTS_SCORE * int(x["genre"] in x["interests"]) + SEX_TYPE_SCORE * int(x["interests_sex"] == x["sex_type"]), axis=1)
# отнормируем скоры так, что бы в среднем на пользователя приходилось ACTION_QUANT действий
user_probs["score"] = user_probs["score"] / user_probs["score"].mean() * ACTION_QUANT / 100
user_probs["click_flag"] = user_probs["score"] > np.random.uniform(size=user_probs.shape[0])
data_click = user_probs[user_probs["click_flag"] == 1]
data_click = data_click[["user_id", "item_id", 'datetime']]
data_click["action"] = ACTIONS[1]
# Добавим даты для последовательных моделей

# Добавим какие объектны нужно было пролистнуть пользователю, что бы увидеть объект на которы йон кликнуд
data_show = data_click.merge(data_windows, on='item_id', how='left')[["user_id","item_showed", "action", 'datetime']].rename(columns={"item_showed": "item_id"})
data_show["action"] = ACTIONS[0]

out = data_click._append(data_show, ignore_index=True)

# Сохраняем результат
out.to_csv('data/interactions.csv', index=False)
users_df.drop(columns=["interests"]).to_csv('data/users.csv', index=False)
items_df.drop(columns=["interests_sex"]).to_csv('data/items.csv', index=False)
user_probs[["user_id", "item_id", "datetime", "score"]].rename(columns={"score": "true_relevance"}).to_csv('data/true_relevance.csv', index=False)
print(out)
print(items_df)

/var/folders/mg/r4cbs1n55z382krlf4zt41z80000gn/T/ipykernel_1140/782987265.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_quality["item_quality"] = item_quality['mean_users_score']
/var/folders/mg/r4cbs1n55z382krlf4zt41z80000gn/T/ipykernel_1140/782987265.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_quality["item_quality"] = item_quality["item_quality"] / item_quality["item_quality"].sum()


        user_id item_id                      datetime action
0             0      11 2025-05-28 07:36:48.225275608  click
1             0      73 2025-07-21 17:26:51.803754354  click
2             0      51 2025-05-05 20:20:44.838313014  click
3             0      74 2025-04-29 05:29:06.051551314  click
4             0      38 2025-04-07 14:57:00.062975580  click
...         ...     ...                           ...    ...
3941443    9999       6 2025-07-12 18:48:25.630890802   show
3941444    9999      29 2025-07-12 18:48:25.630890802   show
3941445    9999      58 2025-07-12 18:48:25.630890802   show
3941446    9999     NaN 2025-07-05 06:57:18.551431996   show
3941447    9999     NaN 2025-04-06 06:03:33.936542636   show

[3941448 rows x 4 columns]
   item_id       genre interests_sex  mean_users_score
0        0     Комедия                        1.872701
1        1       Драма             F          4.753572
2        2   Мелодрама                        3.659970
3        3       Дра

In [6]:
print(out["datetime"].describe())

count                          4033655
mean     2025-05-05 04:25:57.550905344
min      2025-01-01 00:02:54.175952933
25%      2025-03-09 19:59:39.886360064
50%      2025-05-09 15:37:17.440405504
75%      2025-07-02 14:54:23.391997184
max      2025-08-20 11:30:56.342691828
Name: datetime, dtype: object
